Welcome to my fantasy football player prediction system. In this code i will use machine learning to predict how many points a player should get in an upcoming gameweek. I will use a classfier which will produce high accruacy results based on the data which has been trained by a user. 

Firslty we need to load the neccessary packages.

In [318]:
import os
import csv 
import pandas as pd
import glob 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

#Inputting the csv's 

teams = pd.read_csv('PremierLeagueData/2022-23/teams.csv')
teams

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,NaN,1,0,Arsenal,0,0,0,ARS,...,NaN,False,0,1245,1285,1250,1250,1240,1320,1
1,7,0,NaN,2,0,Aston Villa,0,0,0,AVL,...,NaN,False,0,1070,1100,1070,1075,1070,1130,2
2,91,0,NaN,3,0,Bournemouth,0,0,0,BOU,...,NaN,False,0,1035,1095,1020,1110,1050,1080,127
3,94,0,NaN,4,0,Brentford,0,0,0,BRE,...,NaN,False,0,1115,1180,1100,1160,1130,1200,130
4,36,0,NaN,5,0,Brighton,0,0,0,BHA,...,NaN,False,0,1170,1175,1140,1150,1200,1200,131
5,8,0,NaN,6,0,Chelsea,0,0,0,CHE,...,NaN,False,0,1165,1220,1190,1220,1140,1220,4
6,31,0,NaN,7,0,Crystal Palace,0,0,0,CRY,...,NaN,False,0,1085,1100,1110,1110,1060,1090,6
7,11,0,NaN,8,0,Everton,0,0,0,EVE,...,NaN,False,0,1055,1095,1070,1100,1040,1090,7
8,54,0,NaN,9,0,Fulham,0,0,0,FUL,...,NaN,False,0,1095,1100,1070,1065,1120,1140,34
9,13,0,NaN,10,0,Leicester,0,0,0,LEI,...,NaN,False,0,1130,1115,1065,1110,1200,1120,26


In [319]:
directory_path = 'PremierLeagueData/2022-23/understat/'

csv_files = glob.glob(os.path.join(directory_path, '*.csv'))

dataframes = []

for file in csv_files:
    if 'understat_' not in file:
        df = pd.read_csv(file)
        player_name = os.path.basename(file).split('.')[0].replace('_',' ')
        position = df['position']
        
        h_team = df['h_team']
        a_team = df['a_team']
            
        df['player_name'] = player_name
        df['h_team'] = h_team
        df['a_team'] = a_team
        df['position'] = position
        dataframes.append(df)
        
players = pd.concat(dataframes, ignore_index=True)

players = players.groupby(['id', 'player_name', 'position', 'h_team', 'a_team'], as_index=False).agg('sum')

players.to_csv('players.csv', index=False)

print(players)

          id            player_name position             h_team      a_team  \
0         81       Ashley Young 631      AML  Manchester United   Tottenham   
1         81         Ben Davies 660       DL  Manchester United   Tottenham   
2         81  Christian Eriksen 646      AMC  Manchester United   Tottenham   
3         81          Dele Alli 645      Sub  Manchester United   Tottenham   
4         81          Eric Dier 643      DMC  Manchester United   Tottenham   
...      ...                    ...      ...                ...         ...   
58278  19904      Djed Spence 10764       DR             Rennes   Marseille   
58279  19909      Djed Spence 10764       DR            Auxerre      Rennes   
58280  19909   Siriki Dembélé 10749      Sub            Auxerre      Rennes   
58281  19911        Yan Valery 7280       DC             Angers    Toulouse   
58282  19916     Morgan Sanson 3696       MC          Marseille  Strasbourg   

       goals  shots        xG  time  h_goals  a_goa

In [320]:
def predict_players(player_names, players, clf):
    predictions = {}
    for player_name in player_names:
        # filter the data to only include the player's last 5 games
        player_data = players[players['player_name'] == player_name].tail(5)
        
        # preprocess the player's data and make a prediction
        features = dataframes(player_data)
        prediction = clf.predict(features)
        
        # add the prediction to the dictionary
        predictions[player_name] = prediction
        
    return predictions


I will now drop any player that started off as a sub because this will affect the accuracy score and if a player who normally starts but is rested for one week and is started as sub then this won't give an accurate representation.

In [321]:
players = players[players['position'] != 'Sub']
print(players)

          id            player_name position             h_team      a_team  \
0         81       Ashley Young 631      AML  Manchester United   Tottenham   
1         81         Ben Davies 660       DL  Manchester United   Tottenham   
2         81  Christian Eriksen 646      AMC  Manchester United   Tottenham   
4         81          Eric Dier 643      DMC  Manchester United   Tottenham   
5         81         Harry Kane 647       FW  Manchester United   Tottenham   
...      ...                    ...      ...                ...         ...   
58277  19899   Siriki Dembélé 10749      AMC               Nice     Auxerre   
58278  19904      Djed Spence 10764       DR             Rennes   Marseille   
58279  19909      Djed Spence 10764       DR            Auxerre      Rennes   
58281  19911        Yan Valery 7280       DC             Angers    Toulouse   
58282  19916     Morgan Sanson 3696       MC          Marseille  Strasbourg   

       goals  shots        xG  time  h_goals  a_goa

Data Preprocessing & Feature Engieneering

Data Preprocessing - This part will involve me cleaning, transforming and selecting the data which will be relevent for me. 

Feature Engineering- Creating any other bits of data to improve my prediction

Firstly i'm going to separate the goalkeeper, these will need different features to predict the points off as the main focus for there points will be around keeping clean sheets. If a goal keeper keeps a clean sheet they acheive 5 points. 

In [322]:
playersGK = players.copy()
playersGK = playersGK [['player_name', 'time', 'position', 'h_team', 'a_team', 'h_goals', 'a_goals']]
playersGK = playersGK[playersGK['position'].str.startswith('GK')]
playersGK.to_csv('GK_players.csv', index=False)
print(playersGK)

                  player_name  time position            h_team  \
30       Lukasz Fabianski 706    90       GK           Chelsea   
66       Lukasz Fabianski 706    90       GK           Swansea   
71            Hugo Lloris 637    90       GK         Tottenham   
89          Asmir Begovic 694    90       GK   Manchester City   
113      Lukasz Fabianski 706    90       GK        Sunderland   
...                       ...   ...      ...               ...   
57881          Nick Pope 5552    90       GK  Newcastle United   
57899      Fraser Forster 831    90       GK         Tottenham   
57904       Keylor Navas 2243    90       GK         Tottenham   
57919    Alphonse Areola 2310    90       GK          West Ham   
57928  Emiliano Martinez 4401    90       GK          West Ham   

                        a_team  h_goals  a_goals  
30                     Swansea        2        2  
66            Newcastle United        2        0  
71                       Stoke        2        2  
89 

After separeting the goal keepers i'm now going to separate the defenders. I will be introducing goals and assists into this as some defenders reguarly get assists as they are very attacking minded.

In [323]:
playersDEF = players.copy()
playersDEF = playersDEF [['player_name', 'position', 'goals', 'assists', 'shots', 'xG', 'xA', 'time', 'h_team', 'a_team', 'h_goals', 'a_goals', 'key_passes']]
playersDEF = playersDEF[playersDEF['position'].str.startswith('D')]
playersDEF.to_csv('DEF_players.csv', index=False)
print(playersDEF)

             player_name position  goals  assists  shots        xG        xA  \
1         Ben Davies 660       DL      0        0      0  0.000000  0.000000   
4          Eric Dier 643      DMC      0        0      0  0.000000  0.000000   
6        Kyle Walker 638       DR      0        0      1  0.073058  0.000000   
7         Luke Shaw 1006       DL      0        0      0  0.000000  0.000000   
12        Steve Cook 458       DC      0        0      0  0.000000  0.000000   
...                  ...      ...    ...      ...    ...       ...       ...   
58275  Djed Spence 10764       DR      0        0      0  0.000000  0.000000   
58276    Yan Valery 7280       DR      0        0      1  0.086231  0.444642   
58278  Djed Spence 10764       DR      0        0      0  0.000000  0.000000   
58279  Djed Spence 10764       DR      0        0      0  0.000000  0.000000   
58281    Yan Valery 7280       DC      0        0      0  0.000000  0.000000   

       time             h_team       a_

Now i'm separating the midfielders, this will included the same features as the defenders because if a midfielder keeps a clean sheet they earn an extra point. 

In [324]:
playersMID = players.copy()
playersMID = playersMID [['player_name', 'position', 'goals', 'assists', 'shots', 'xG', 'xA', 'time', 'h_team', 'a_team', 'h_goals', 'a_goals', 'key_passes']]
playersMID = playersMID[playersMID['position'].str.startswith('M')]
playersMID.to_csv('MID_players.csv', index=False)
print(playersMID)

               player_name position  goals  assists  shots        xG  \
9        Idrissa Gueye 668       MC      0        0      2  0.131937   
11        Matt Ritchie 461       MR      0        0      2  0.042670   
17     Marc Albrighton 753       MR      1        2      2  0.155681   
19        Riyad Mahrez 750       ML      2        0      6  1.246980   
20      James McArthur 633       MC      0        1      0  0.000000   
...                    ...      ...    ...      ...    ...       ...   
58260   Morgan Sanson 3696       ML      0        0      0  0.000000   
58263   Morgan Sanson 3696       ML      0        1      0  0.000000   
58269   Morgan Sanson 3696       MC      0        0      5  0.450222   
58273   Morgan Sanson 3696       MC      0        0      0  0.000000   
58282   Morgan Sanson 3696       MC      0        0      4  0.263555   

             xA  time         h_team          a_team  h_goals  a_goals  \
9      0.000000    90    Bournemouth     Aston Villa        0

Finally i'm seperating the forwards which won't include goals condeded as if a forwards team concedes they don't lose any points. The main feature for these will be goals scored and assists becuase that will be where the majority of their points come from. 

In [325]:
playersFWD = players.copy()
playersFWD = playersFWD [['player_name', 'position', 'goals', 'assists', 'shots', 'xG', 'xA', 'time', 'h_team', 'a_team', 'h_goals', 'a_goals', 'key_passes']]
playersFWD = playersFWD[playersFWD['position'].str.startswith('F')]
playersFWD.to_csv('FWD_players.csv', index=False)
print(playersFWD)

               player_name position  goals  assists  shots        xG  \
5           Harry Kane 647       FW      0        0      3  0.182191   
8        Callum Wilson 468       FW      0        0      2  0.131566   
10         Jordan Ayew 672      FWR      0        0      0  0.000000   
15         Jamie Vardy 755       FW      1        0      5  0.774049   
27         Diego Costa 802       FW      0        0      1  0.050366   
...                    ...      ...    ...      ...    ...       ...   
58186           Tetê 10536      FWR      0        0      0  0.000000   
58190           Tetê 10536      FWR      0        0      6  0.664827   
58225  Dango Ouattara 9662       FW      1        0      1  0.132618   
58235           Tetê 10536      FWR      0        0      2  0.222425   
58251           Tetê 10536      FWR      0        0      2  0.077242   

             xA  time             h_team       a_team  h_goals  a_goals  \
5      0.297304    90  Manchester United    Tottenham       

Model Selection- Selecting my ML algorithm which is most appropriate and achieves the highest accuracy. 

In [473]:
import pandas as pd

# Load the original dataset
df = pd.read_csv('players.csv')

# Select the relevant columns
columns = ['id', 'player_name', 'goals', 'shots', 'xG', 'assists', 'season']
df = df[columns]

# Sort the dataset by player_name and season
df = df.sort_values(['player_name', 'season'])

# Use a sliding window approach to create the input features
window_size = 5
X = []
y = []

for name, group in df.groupby('player_name'):
    for i in range(window_size, len(group)):
        # Get the last 5 games for the current player
        last_five = group.iloc[i - window_size:i]

        # Get the features and target for the current game
        current_game = group.iloc[i]
        features = last_five[['goals', 'shots', 'xG', 'assists']].values.flatten()
        target = current_game['goals']

        # Append the features and target to the input and output arrays
        X.append(features)
        y.append(target)

X = np.array(X)
y = np.array(y)


In [474]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000)
clf.fit(X_train_scaled, y_train)

# Evaluate the model
score = clf.score(X_test_scaled, y_test)
print('Accuracy:', score)


Accuracy: 0.8980269058295964


In [454]:
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the data
df = pd.read_csv('players.csv')

# Select the relevant columns
cols = ['player_name', 'goals', 'shots', 'assists']

# Get the last 5 games for each player
players_last_5_games = df[cols].groupby('player_name').apply(lambda x: x[-5:]).reset_index(drop=True)

# Aggregate the stats for each player
players_stats = players_last_5_games.groupby('player_name').agg({'goals': 'sum', 'shots': 'sum', 'assists': 'sum'})

# Set the target variable to be the goals scored in the next game
players_stats['target'] = df.groupby('player_name')['goals'].apply(lambda x: x.iloc[-1])

# Display the players_stats dataframe
players_stats.head()

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(players_stats.drop('target', axis=1), players_stats['target'], test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
regressor = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000)
regressor.fit(X_train_scaled, y_train)

# Make a prediction for a random player
import random
player_name = random.choice(players_stats.index)
player_stats = players_stats.loc[player_name].drop('target')
player_stats_scaled = scaler.transform(player_stats.values.reshape(1, -1))
goals_predicted = regressor.predict(player_stats_scaled)[0]

print(f"{player_name}: Goals predicted: {goals_predicted:.2f}")


Phil Foden 6055: Goals predicted: 0.38


C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [389]:
import pandas as pd

from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Load the data
df = pd.read_csv('players.csv')

player_type = 'forward'

# Select the relevant columns
cols = ['player_name', 'goals', 'shots', 'xG', 'assists', 'xA', 'h_team', 'a_team', 'h_goals', 'a_goals', 'position']

# Get the last 5 games for each player
players_last_5_games = df[cols].groupby('player_name').apply(lambda x: x[-5:]).reset_index(drop=True)

# Assign each player to a team based on their last 5 games
player_team_counts = players_last_5_games.groupby(['player_name', 'h_team']).size().add(players_last_5_games.groupby(['player_name', 'a_team']).size(), fill_value=0)
player_teams = player_team_counts[player_team_counts >= 3].groupby('player_name').apply(lambda x: x.idxmax()[1]).reset_index(name='team_name')

# Filter the last 5 games to only include players with assigned teams
players_last_5_games = pd.merge(players_last_5_games, player_teams, on='player_name')
players_last_5_games = players_last_5_games[players_last_5_games['h_team'] == players_last_5_games['team_name']]

# Filter to only include players of the desired type (forward or goalkeeper)
if player_type == 'forward':
    players_last_5_games = players_last_5_games[players_last_5_games['position'] != 'GK']
elif player_type == 'goalkeeper':
    players_last_5_games = players_last_5_games[players_last_5_games['position'] == 'GK']

# Aggregate the stats for each player
players_stats = players_last_5_games.groupby('player_name').agg({'goals': 'sum', 'shots': 'sum', 'xG': 'sum', 'assists': 'sum', 'xA': 'sum', 'h_goals': 'sum'})

# Set the target variable to be the goals scored in the next game for forwards and the goals conceded in the next game for goalkeepers
temp = df.groupby('player_name')['a_goals'].apply(lambda x: x.iloc[-1]).reset_index()
temp = temp.rename(columns={'a_goals': 'target'})
players_stats = players_stats.merge(temp, on='player_name', how='left')

# Filter goalkeepers and merge with players_stats
goalkeeper_stats = players_stats.loc[players_stats.index.isin(players_last_5_games.loc[players_last_5_games['position'] == 'GK']['player_name'])]
goalkeeper_stats = goalkeeper_stats.rename(columns={'a_goals': 'target'})
players_stats.update(goalkeeper_stats)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(players_stats.drop(['target', 'player_name'], axis=1), players_stats['target'], test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50), (100,100)],
#     'activation': ['logistic', 'tanh', 'relu'],
#     'solver': ['adam', 'sgd'],
#     'learning_rate': ['constant', 'adaptive'],
#     'alpha': [0.0001, 0.001, 0.01, 0.1],
# }

# Train the model
model = MLPRegressor(hidden_layer_sizes=(50, 50), activation='logistic', solver='sgd', alpha=0.1, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=500, shuffle=True, random_state=42, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.fit(X_train_scaled, y_train)

# # # Perform grid search
# grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train_scaled, y_train)

# # # Get the best parameters
# best_params = grid_search.best_params_
# print("Best parameters:", best_params)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Add the player_name column back to the testing set
X_test['player_name'] = players_stats.loc[X_test.index, 'player_name']

# Combine the testing set with the predicted target values
predictions_df = X_test.copy()
predictions_df['target'] = y_pred

# Print out the predicted values along with the corresponding player names
print(predictions_df[['player_name', 'target']])

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)

print("Mean squared error: ", mse)
print("Mean absolute error: ", mae)
print("R-squared value: ", r2)

                     player_name    target
93           Dango Ouattara 9662  1.243078
320              Max Kilman 7332  1.227677
55          Bruno Fernandes 1228  1.301916
30                  Antony 11094  1.228642
449  Trent Alexander-Arnold 1791  1.243947
..                           ...       ...
249           Kaoru Mitoma 10806  1.234286
393              Robin Koch 6273  1.226711
425         Siriki Dembélé 10749  1.231191
42                  Ben Mee 1654  1.224695
228          Jordan Zemura 10740  1.228427

[96 rows x 2 columns]
Mean squared error:  1.1059178591104635
Mean absolute error:  0.8574113421948629
R-squared value:  -0.019315830539257117


In [ ]:
#GRID SEARCH

# param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50), (100,100)],
#     'activation': ['logistic', 'tanh', 'relu'],
#     'solver': ['adam', 'sgd'],
#     'learning_rate': ['constant', 'adaptive'],
#     'alpha': [0.0001, 0.001, 0.01, 0.1],
# }


# # # Perform grid search
# grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train_scaled, y_train)

# # # Get the best parameters
# best_params = grid_search.best_params_
# print("Best parameters:", best_params)

In [550]:
# Filter the data to get the last 10 games for each player
last_10_games = df.groupby("player_name").tail(10)

# Create the y matrix for goals, shots, and assists
y = last_10_games[["goals", "shots", "assists"]]

# Print the y matrix
print(y.shape)

(4920, 3)


In [552]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("players.csv")

# Filter the data to get the last 10 games for each player
last_10_games = df.groupby("player_name").tail(10)

# Select the features for X
X = last_10_games[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]]

# Create the y matrix for goals, shots, and assists
y = last_10_games[["goals", "shots", "assists"]]

# Scale the features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Print the shape of X
print(X.shape)

# Print the y matrix
print(y.shape)

(4920, 12)
(4920, 3)


In [567]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("players.csv")

# Filter the data to get the last 10 games for each player
last_10_games = df.groupby("player_name").tail(10)

# Select the features for X
X = last_10_games[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]]

# Create the y matrix for goals, shots, and assists
y = last_10_games[["goals", "shots", "assists"]]

# Scale the features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Print the shape of X
print(X.shape)

# Print the y matrix
print(y.shape)

# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest regressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regressor.predict(X_test)

# Make predictions for 10 random players
import random
random.seed(0)
for i in range(10):
    player_id = random.choice(df['player_name'].unique())
    player_data = player_data.reshape(1, -1)
    y_pred_next = regressor.predict(player_data)
    print(f"Player {player_id} is predicted to score {float(y_pred_next[0][0]):.2f} goals in their next game.")

# Calculate the root mean squared error (RMSE) of the predictions
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

from sklearn.metrics import r2_score

# Calculate the R-squared score of the predictions
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

(4920, 12)
(4920, 3)
Player Emmanuel Dennis 9301 is predicted to score 0.01 goals in their next game.
Player Moisés Caicedo 9453 is predicted to score 0.01 goals in their next game.
Player Theo Walcott 503 is predicted to score 0.01 goals in their next game.
Player Willy Boly 6850 is predicted to score 0.01 goals in their next game.
Player Jhon Durán 11366 is predicted to score 0.01 goals in their next game.
Player Juan Larios 11078 is predicted to score 0.01 goals in their next game.
Player Michael Olise 9948 is predicted to score 0.01 goals in their next game.
Player Carlos Vinicius 7395 is predicted to score 0.01 goals in their next game.
Player Antony 11094 is predicted to score 0.01 goals in their next game.
Player Kalvin Phillips 8719 is predicted to score 0.01 goals in their next game.
RMSE: 0.10608670439984495
R-squared score: 0.7052225722662714


In [627]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np

df = pd.read_csv("players.csv")

#dropping any player who's starting position is sub
df = df[df['position'] != 'Sub']

# Filter the data to get the last 10 games for each player
last_10_games = df.groupby("player_name").tail(10)

# Select the features for X
X = last_10_games[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]]

# Create the y matrix for goals only
y = last_10_games[["goals"]]

# Encode the position column using LabelEncoder
le = LabelEncoder()
last_10_games["position_encoded"] = le.fit_transform(last_10_games["position"])

# Scale the features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X.values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest regressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regressor.predict(X_test)

# Create an empty list to store predicted values for each player
preds_list = []

for player_id in df['player_name'].unique():
    player_data = df[df['player_name'] == player_id].tail(10)  # select last 10 games for each player
    X = scaler.transform(player_data[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]].values)
    
    y = player_data['goals']
    pred = regressor.predict(X)
    rmse = np.sqrt(mean_squared_error(y, pred))
    r2 = r2_score(y, pred)
    # Append the predicted value to the preds_list
    preds_list.append({'player_name': player_id, 'predicted_goals': round(pred[0], 2)})

top_predicted = preds.sort_values(by='predicted_goals', ascending=False)
print(top_predicted.head(10))

# Create a dataframe from the preds_list
preds = pd.DataFrame(preds_list)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)

print("Mean squared error: ", mse)
print("Mean absolute error: ", mae)
print("R-squared value: ", r2)

C:\Users\aaron\AppData\Local\Temp\ipykernel_260\2500154015.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_games["position_encoded"] = le.fit_transform(last_10_games["position"])
C:\Users\aaron\AppData\Local\Temp\ipykernel_260\2500154015.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)
C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\

               player_name  predicted_goals
265        Phil Foden 6055             2.86
392         Ivan Toney 998             2.02
459   Julián Álvarez 10846             2.00
17        Riyad Mahrez 750             2.00
334     Taiwo Awoniyi 7814             1.98
208   Dominic Solanke 1679             1.00
317     Eddie Nketiah 6482             1.00
198       Solly March 6049             1.00
430             Tetê 10536             1.00
72   Kelechi Iheanacho 620             1.00
Mean squared error:  1.295896328293739e-06
Mean absolute error:  6.479481641468689e-05
R-squared value:  0.9999885932495689


C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [629]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf

df = pd.read_csv("players.csv")

#dropping any player who's starting position is sub
df = df[df['position'] != 'Sub']

# Filter the data to get the last 10 games for each player
last_10_games = df.groupby("player_name").tail(10)

# Select the features for X
X = last_10_games[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]]

# Create the y matrix for goals only
y = last_10_games[["goals"]]

# Encode the position column using LabelEncoder
le = LabelEncoder()
last_10_games["position_encoded"] = le.fit_transform(last_10_games["position"])

# Scale the features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X.values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the architecture of your MLP
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model on your training data
model.fit(X_train, y_train, epochs=50, batch_size=16)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Create an empty list to store predicted values for each player
preds_list = []

for player_id in df['player_name'].unique():
    player_data = df[df['player_name'] == player_id].tail(10)  # select last 10 games for each player
    X = scaler.transform(player_data[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]].values)
    
    y = player_data['goals']
    pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, pred))
    r2 = r2_score(y, pred)
    # Append the predicted value to the preds_list
    preds_list.append({'player_name': player_id, 'predicted_goals': round(pred[0][0], 2)})

top_predicted = pd.DataFrame(preds_list).sort_values(by='predicted_goals', ascending=False)
print(top_predicted.head(10))

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)

print("Mean squared error: ", mse)
print("Mean absolute error: ", mae)
print("R-squared value: ", r2)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# X["position_encoded"] = le.transform(player_data["position"])
    # position_dummies = pd.get_dummies(X['position_encoded'])
    # X = pd.concat([X, position_dummies], axis=1)
    
    # # Create a new StandardScaler object to scale the updated X
    # scaler = StandardScaler()
    # X = scaler.fit_transform(X.values)

In [569]:
def make_predictions(model, data):
    """
    Make predictions using the trained model.

    Args:
    model: A trained linear regression model.
    data: A Pandas dataframe containing the data to make predictions on.

    Returns:
    A Pandas dataframe containing the player IDs and their predicted number of goals.
    """
    # Create a new dataframe to hold the predicted values
    preds = pd.DataFrame(columns=['player_id', 'predicted_goals'])

    # Loop over each player in the dataset and make a prediction for them
    for player_id in data['player_id'].unique():
        player_data = data[data['player_id'] == player_id]
        X = player_data.drop(['player_id', 'goals'], axis=1)
        y = player_data['goals']
        pred = model.predict(X)
        rmse = np.sqrt(mean_squared_error(y, pred))
        r2 = r2_score(y, pred)
        # Append the predicted value to the preds dataframe
        preds = preds.append({'player_id': player_id, 'predicted_goals': pred[0]}, ignore_index=True)
        print(f"Player {player_id} is predicted to score {pred[0]:.2f} goals in their next game.")
        print(f"RMSE: {rmse}")
        print(f"R-squared score: {r2}")
    
    return preds

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load data into pandas dataframe
data = pd.read_csv('players.csv')

# Calculate the average goals scored by each player in their last 10 games
last_10 = data.groupby('player_id').tail(10)
average_goals = last_10.groupby('player_id')['goals'].mean().reset_index()

# Train a linear regression model to predict goals based on minutes played
model = LinearRegression()
model.fit(data[['minutes']], data[['goals']])

# Make predictions for each player based on their average minutes played in the last 10 games
for index, row in average_goals.iterrows():
    player_id = row['player_id']
    minutes = last_10[last_10['player_id'] == player_id]['minutes'].mean()
    predicted_goals = model.predict([[minutes]])
    print(f"Player {player_id} is predicted to score {predicted_goals[0][0]:.2f} goals in their next game.")

In [532]:
from sklearn.metrics import r2_score

# Calculate the R-squared score of the predictions
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)


R-squared score: 0.7052225722662714


In [546]:
import random

# Select a random player from the dataset
player_id = random.choice(df['player_name'].unique())

player_id = player_id[player_id['position'] != 'Sub']

# Extract the last 10 games of the player
player_games = df[df['player_name'] == player_id].tail(10)

# Prepare the input data for the model
X = player_games.iloc[:, 2:5].values  # minutes played, assists, yellow cards

# Make a prediction for the player's next game
y_pred = regressor.predict(X)[-1]

print(f"Player {player_id} is predicted to score {y_pred:.2f} goals in their next game.")

TypeError: string indices must be integers

In [486]:
import pandas as pd

from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('players.csv')

player_type = 'forward'

# Select the relevant columns
cols = ['player_name', 'goals', 'shots', 'assists', 'h_team', 'a_team', 'h_goals', 'a_goals', 'position']

# Get the last 5 games for each player
players_last_5_games = df[cols].groupby('player_name').apply(lambda x: x[-5:]).reset_index(drop=True)

# Assign each player to a team based on their last 5 games
player_team_counts = players_last_5_games.groupby(['player_name', 'h_team']).size().add(players_last_5_games.groupby(['player_name', 'a_team']).size(), fill_value=0)
player_teams = player_team_counts[player_team_counts >= 3].groupby('player_name').apply(lambda x: x.idxmax()[1]).reset_index(name='team_name')

# Filter the last 5 games to only include players with assigned teams
players_last_5_games = pd.merge(players_last_5_games, player_teams, on='player_name')
players_last_5_games = players_last_5_games[players_last_5_games['h_team'] == players_last_5_games['team_name']]

# Filter to only include players of the desired type (forward or goalkeeper)
if player_type == 'forward':
    players_last_5_games = players_last_5_games[players_last_5_games['position'] != 'GK']
elif player_type == 'goalkeeper':
    players_last_5_games = players_last_5_games[players_last_5_games['position'] == 'GK']
    # Compute the target variable for goalkeepers as the number of goals conceded in the next game
    temp = df.groupby('player_name')['h_goals'].apply(lambda x: x.iloc[-1]).reset_index()
    temp = temp.rename(columns={'h_goals': 'target'})
    players_last_5_games = pd.merge(players_last_5_games, temp, on='player_name', how='left')

# Aggregate the stats for each player
players_stats = players_last_5_games.groupby('player_name').agg({'goals': 'sum', 'shots': 'sum', 'assists': 'sum', 'h_goals': 'sum'})

# Set the target variable to be the goals scored in the next game for forwards and the goals conceded in the next game for goalkeepers
temp = df.groupby('player_name')['a_goals'].apply(lambda x: x.iloc[-1]).reset_index()
temp = temp.rename(columns={'a_goals': 'target'})
players_stats = players_stats.merge(temp, on='player_name', how='left')

# Filter goalkeepers and merge with players_stats
goalkeeper_stats = players_stats.loc[players_stats.index.isin(players_last_5_games.loc[players_last_5_games['position'] == 'GK']['player_name'])]
goalkeeper_stats = goalkeeper_stats.rename(columns={'a_goals': 'target'})
players_stats.update(goalkeeper_stats)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(players_stats.drop(['target', 'player_name'], axis=1), players_stats['target'], test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
model = MLPRegressor(hidden_layer_sizes=(50, 50), activation='logistic', solver='sgd', alpha=0.1, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=500, shuffle=True, random_state=42, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Add the player_name column back to the testing set
X_test['player_name'] = players_stats.loc[X_test.index, 'player_name']

# Combine the testing set with the predicted target values
predictions_df = X_test.copy()
predictions_df['target'] = y_pred

def assign_points(predictions_df):
    points = []
    for i, row in predictions_df.iterrows():
        if row['target'] >= 1.4:
            points.append(10)
        elif row['target'] >= 1.3:
            points.append(7)
        elif row['target'] >= 1.2:
            points.append(5)
        elif row['target'] >= 1.1:
            points.append(3)
        else:
            points.append(0)
    predictions_df['points'] = points
    return predictions_df

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Add the player_name column back to the testing set
X_test['player_name'] = players_stats.loc[X_test.index, 'player_name']

# Combine the testing set with the predicted target values
predictions_df = X_test.copy()
predictions_df['target'] = y_pred

# Assign points to each player based on their predicted performance
predictions_df = assign_points(predictions_df)

# Print out the predicted values, along with the corresponding player names and points
print(predictions_df[['player_name', 'target', 'points']])

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)

print("Mean squared error: ", mse)
print("Mean absolute error: ", mae)
print("R-squared value: ", r2)

                     player_name    target  points
93           Dango Ouattara 9662  1.196938       3
320              Max Kilman 7332  1.204116       5
55          Bruno Fernandes 1228  1.193125       3
30                  Antony 11094  1.199511       3
449  Trent Alexander-Arnold 1791  1.195499       3
..                           ...       ...     ...
249           Kaoru Mitoma 10806  1.194264       3
393              Robin Koch 6273  1.201549       5
425         Siriki Dembélé 10749  1.203074       5
42                  Ben Mee 1654  1.194039       3
228          Jordan Zemura 10740  1.203074       5

[96 rows x 3 columns]
Mean squared error:  1.096064460094544
Mean absolute error:  0.8435939095024922
R-squared value:  -0.010234029826114321


In [488]:
print(y.shape)

(55748,)


In [487]:
print(goalkeeper_stats.columns)

Index(['player_name', 'goals', 'shots', 'assists', 'h_goals', 'target'], dtype='object')


Model Training- Train the model selected using different type of parameters which can improve it's performance.

Model Evaluation- Evaluate the performance of the model by testing.  